## Reading 33-2 - Thread Scheduling Basics

The OS determines <b>how long a thread should run</b> on a core and <b>which thread should run next</b>.

For the purposes of this lecture, let’s assume that a thread will run on a core <b>until its program terminates</b> or it is <b>forced off the processor</b>.

> There are many reasons why a thread may be booted from a core: sometimes the operating system deems a thread needs to vacate its spot, and other times a thread will voluntarily yield its core.


### Code Example 

So how do simple threads work? First, we need to include two libraries: <a href = "https://man7.org/linux/man-pages/man0/pthread.h.0p.html">pthread.h</a> and <a href = "https://man7.org/linux/man-pages/man0/unistd.h.0p.html">unistd.h</a>

<b><code>pthread.h</code></b> - POSIX thread, which means <a href = "https://en.wikipedia.org/wiki/POSIX">Portable Operating System Interface</a>, the IEEE standard for maintaining compatibility between operating systems.
<ul><li>Allows you to create, update, and destroy threads in a program</li></ul>

<b><code>unistd.h</code></b> allows us to interface with the POSIX Application Programming Interface (API)

#### pthread_1.cpp

In our code at <a href = "">pthread_1.cpp</a>, the first thing we do is initialize the pthread with the type pthread_t, and a variable tid_0 (meaning thread ID #0). 

> The id is a long unsigned integer, and we can now see that the data is located at a specific location

    #include <iostream>
    #include <cstdlib>
    #include <unistd.h>     // Interface with the POSIX API
    #include <pthread.h>    // Pthread library

    int main(){

        // Created, but not initialized - Will not pass PQC yet
        pthread_t tid_0;
        std::cout << &tid_0 << " " << tid_0 << std::endl;

        return EXIT_SUCCESS;
    }
    
Since the pthread is not initialized, it will not pass PQC, so our Makefile does not include them for make pthread_1. (They are included for the rest of the commands) You can see that memory is allocated, but there is no ID number yet.


    > make pthread_1 
    gcc -std=c++2a -c pthread_1.c
    gcc -std=c++2a -o pthread_1 pthread_1.o 
    > ./pthread_1 
    0x7fff2ff6e478 0 

### The Nine Steps for Writing a valid pthread

I will first state the Nine steps for writing a basic multithreaded program. We will practice a lot in Lecture.

> These steps do not account for accounting for race conditions or mutual exclusion, which we will study later this week.

The pthread example where I complete all nine steps may be found at <a href = "">pthread_2.cpp</a>
<ol>
    <li>Write a struct representing the inputs to a function you wish to parallelize</li>
    <li>Write the function you wish to implement, except pass a <code>void</code> pointer to the function, and return a <code>void</code> pointer.</li>
    <li>Cast the <code>void*</code> back to a struct pointer inside the function</li>
    <li>You must return a <code>void</code> pointer in the function, so you will write <code>return NULL;</li>
    <li>Create a dynamically allocated struct object in main, and set the values you'd want to input to the function</li>
    <li>Call <code>pthread_create</code>, calling the <code>thread_ID</code> by reference, <code>NULL</code> and then pass the struct to the pthread by casting to a <code>void</code> pointer</li>
    <li>Join the pthreads calling <code>pthread_join</code></li>
    <li>Free the dynamically allocated structs for the function inputs</li>
    <li>Call <code>pthread_exit( NULL );</code> as the last thing you do before <code>return</code> to main</li>
</ol>

### Examples of the Steps for Writing a pthread

#### Step 1 - Write a struct representing the inputs to a function you wish to parallelize 

Let's say we want to take in the pthread_id and two integers, your struct would look like this:

    typedef struct thread_func_args{

        pthread_t curr_tid;
        int input_1;
        int input_2;

    }thread_func_args;


#### Step 2 - Write the function you wish to implement, except pass a void pointer to the function, and return a void pointer.

The function you will call will look like this:

    void* thread_func( void* inputs )

    
#### Step 3 - Cast the void* back to a struct pointer inside the function

When you call the function, you cast inputs to a <code>thread_func_args</code> struct, which allows you to access the memory

    thread_func_args* func_inputs = (thread_func_args *)inputs;

Then you access the data by de-referencing the inputs:

    std::cout << func_inputs->input_1 << " " << func_inputs->input_2 << std::endl;

  
#### Step 4 - You must return a void pointer in the function, so you will write return NULL;

    return NULL;   

   
#### Step 5 - Create a dynamically allocated struct object in main, and set the values you'd want to input to the function

In main, you will set up the struct pointer and set the values

    thread_func_args* tid_0_inputs = (thread_func_args*)malloc( sizeof(thread_func_args) );
    tid_0_inputs->input_1 = 10;
    tid_0_inputs->input_2 = 20;

   

#### Step 6 - Call pthread_create, calling the thread_ID by reference, NULL (we are not adding attributes in this course) and then pass the struct to the pthread by casting to a void pointer

Here is how to call the <code>pthread_create</code> function:
* first argument is to pass the thread ID by reference. Since this is C, we will pass the address of the pthread.

But there are more inputs, which allow us to create the entire thread. The second input is a data structure of attributes that you can assign to the new thread you are creating. These attributes describe the behavior of that thread.

For example, we could tell the thread to end at a different time than main.
But we haven't studied Operating Systems yet, so for now we are just going to pass NULL as the second input.

The third thread is important. It is a pointer to the function that you want this new thread to execute. When you create a thread, you must give it a piece of code you can execute. For this example, we will call the function thread_func.


    pthread_create( &(tid_0_inputs->curr_tid), NULL, thread_func, (void *)tid_0_inputs );

 

#### Step 7 - Join the pthreads calling pthread_join

Pass the thread ID, which in this case is tid_0_inputs->curr_tid, and then NULL

    pthread_join( tid_0_inputs->curr_tid,  NULL );

   

#### Step 8 - Free the dynamically allocated structs for the function inputs

    free( tid_0_inputs );

   

#### Step 9 - Call pthread_exit( NULL ); as the last thing you do before return main

This will force main to wait until all the threads initialized in main are finished, otherwise main will finish too fast.

    pthread_exit( NULL );

###  <font color = "red">Class Introduction Question #3 - Describe Step 1 for writing a pthread.</a>

###  <font color = "red">Class Introduction Question #4 - Describe Step 2 for writing a pthread.</a>

###  <font color = "red">Class Introduction Question #5 - Describe Step 3 for writing a pthread.</a>

###  <font color = "red">Class Introduction Question #6 - Describe Step 4 for writing a pthread.</a>

###  <font color = "red">Class Introduction Question #7 - Describe Step 5 for writing a pthread.</a>

###  <font color = "red">Class Introduction Question #8 - Describe Step 6 for writing a pthread.</a>

###  <font color = "red">Class Introduction Question #9 - Describe Step 7 for writing a pthread.</a>

###  <font color = "red">Class Introduction Question #10 - Describe Step 8 for writing a pthread.</a>

###  <font color = "red">Class Introduction Question #11 - Describe Step 9 for writing a pthread.</a>